In [32]:
#Подключим нужные библиотеки
import numpy as np
import pandas as pd

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.model_selection import RandomizedSearchCV

from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from catboost import CatBoostClassifier
from sklearn.metrics import classification_report, accuracy_score

from scipy.stats import randint, uniform

Так как ранее данные уже были проанализированны и очишенны от выбросов и неэффективгых признаков сразу приступаем к построению моделей.

In [10]:
#Загрузи данные
IC50_df = pd.read_csv('./data/IC50.csv')
display(IC50_df)

,Unnamed: 0,"IC50, mM",MaxEStateIndex,MinAbsEStateIndex,MinEStateIndex,qed,SPS,MaxPartialCharge,MaxAbsPartialCharge,FpDensityMorgan1,...,fr_lactone,fr_nitro,fr_piperdine,fr_priamide,fr_pyridine,fr_quatN,fr_sulfide,fr_tetrazole,fr_thiazole,fr_unbrch_alkane
0,0,6.239374,5.094096,0.387225,0.387225,0.417362,42.928571,0.038844,0.293526,0.642857,...,0,0,0,0,0,0,0,0,0,3
1,1,0.771831,3.961417,0.533868,0.533868,0.462473,45.214286,0.012887,0.313407,0.607143,...,0,0,0,0,0,0,0,0,0,3
2,2,223.808778,2.627117,0.543231,0.543231,0.260923,42.187500,0.094802,0.325573,0.562500,...,0,0,0,0,0,2,0,0,0,3
3,3,1.705624,5.097360,0.390603,0.390603,0.377846,41.862069,0.038844,0.293526,0.620690,...,0,0,0,0,0,0,0,0,0,4
4,4,107.131532,5.150510,0.270476,0.270476,0.429038,36.514286,0.062897,0.257239,0.600000,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
885,996,31.000104,12.934891,0.048029,-0.476142,0.382752,49.133333,0.317890,0.468587,1.133333,...,2,0,0,0,0,0,0,0,0,0
886,997,31.999934,13.635345,0.030329,-0.699355,0.369425,44.542857,0.327562,0.467493,1.085714,...,2,0,0,0,0,0,0,0,0,0
887,998,30.999883,13.991690,0.026535,-0.650790,0.284923,41.973684,0.327887,0.467485,1.157895,...,2,0,0,0,0,0,1,0,0,0
888,999,31.998959,13.830180,0.146522,-1.408652,0.381559,39.000000,0.312509,0.468755,0.756757,...,0,0,0,0,0,0,0,0,0,0


Преобразим целеавой признак в True и False для выполнения классификации.

Значение True установим если значение IC50 > медианного значения иначе установим False.

Так как медианное значение это значение центрального элемента выборки, то имеем сбалансированный датасет для обучения.

In [11]:
#Преобразим целевой признак
target = IC50_df['IC50, mM'] >= IC50_df['IC50, mM'].median()

In [12]:
#Выведем признаки
features = IC50_df.drop(['IC50, mM', 'Unnamed: 0'], axis=1)

In [13]:
#Выполним стандартизацию данных
scaler = StandardScaler()
features = scaler.fit_transform(features)

In [14]:
#Разделение на обучающую и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=42)

<h2>Модель логистической регрессии</h2>

In [45]:
model = LogisticRegression(
    max_iter=200
)
model.fit(X_train, y_train)

# Предсказание
y_pred = model.predict(X_test)
print('Логистическая регрессия:')
print(f'Accuracy: {accuracy_score(y_test, y_pred)}')
print(classification_report(y_test, y_pred))

Логистическая регрессия:
Accuracy: 0.6966292134831461
              precision    recall  f1-score   support

       False       0.76      0.63      0.69        94
        True       0.65      0.77      0.71        84

    accuracy                           0.70       178
   macro avg       0.70      0.70      0.70       178
weighted avg       0.71      0.70      0.70       178



<h2>Модель DecisionTree</h2>

In [58]:
model = DecisionTreeClassifier(
    max_depth=7,
    min_samples_split=2,
    min_samples_leaf=1,
    random_state=42
)
model.fit(X_train, y_train)

# Предсказание
y_pred = model.predict(X_test)
print('Дерево решений:')
print(f'Accuracy: {accuracy_score(y_test, y_pred)}')
print(classification_report(y_test, y_pred))

Дерево решений:
Accuracy: 0.7247191011235955
              precision    recall  f1-score   support

       False       0.74      0.73      0.74        94
        True       0.71      0.71      0.71        84

    accuracy                           0.72       178
   macro avg       0.72      0.72      0.72       178
weighted avg       0.72      0.72      0.72       178



<h2>Модель Random Forest</h2>

In [99]:
#Обучение модели
model = RandomForestClassifier(
    n_estimators=75,
    max_depth=8,
    random_state=42
)
model.fit(X_train, y_train)

# Предсказание
y_pred = model.predict(X_test)
print('Ансамбль деревьев:')
print(f'Accuracy: {accuracy_score(y_test, y_pred)}')
print(classification_report(y_test, y_pred))

Ансамбль деревьев:
Accuracy: 0.7640449438202247
              precision    recall  f1-score   support

       False       0.79      0.76      0.77        94
        True       0.74      0.77      0.76        84

    accuracy                           0.76       178
   macro avg       0.76      0.76      0.76       178
weighted avg       0.77      0.76      0.76       178



<h2>Модель SVС</h2>

In [76]:
# Обучение модели
model = SVC(
    kernel='rbf',
    C=150
)
model.fit(X_train, y_train)

# Предсказание
y_pred = model.predict(X_test)
print('Классивикация по опорным векторам:')
print(f'Accuracy: {accuracy_score(y_test, y_pred)}')
print(classification_report(y_test, y_pred))

Классивикация по опорным векторам:
Accuracy: 0.6966292134831461
              precision    recall  f1-score   support

       False       0.71      0.72      0.72        94
        True       0.68      0.67      0.67        84

    accuracy                           0.70       178
   macro avg       0.70      0.70      0.70       178
weighted avg       0.70      0.70      0.70       178



<h2>Модель KNN</h2>

In [87]:
# Обучение модели
model = KNeighborsClassifier(
    n_neighbors=5,
    weights='distance'
)
model.fit(X_train, y_train)

# Предсказание
y_pred = model.predict(X_test)
print('Регрессия по K-ближайшим соседям:')
print(f'Accuracy: {accuracy_score(y_test, y_pred)}')
print(classification_report(y_test, y_pred))

Регрессия по K-ближайшим соседям:
Accuracy: 0.7415730337078652
              precision    recall  f1-score   support

       False       0.77      0.72      0.75        94
        True       0.71      0.76      0.74        84

    accuracy                           0.74       178
   macro avg       0.74      0.74      0.74       178
weighted avg       0.74      0.74      0.74       178



In [88]:
model = catboost = CatBoostClassifier(
    iterations=100,
    verbose=0,
    random_state=42
)

# Обучение модели
model.fit(X_train, y_train, eval_set=(X_test, y_test))

# Предсказание
y_pred = model.predict(X_test)
print('Регрессия CatBoost:')
print(f'Accuracy: {accuracy_score(y_test, y_pred)}')
print(classification_report(y_test, y_pred))

Регрессия CatBoost:
Accuracy: 0.7696629213483146
              precision    recall  f1-score   support

       False       0.78      0.79      0.78        94
        True       0.76      0.75      0.75        84

    accuracy                           0.77       178
   macro avg       0.77      0.77      0.77       178
weighted avg       0.77      0.77      0.77       178



<b>Для лучшей модели попробуем автоматический подбор гипер параметров</b>

In [94]:
param_dist = {
    'n_estimators': randint(3, 100),
    'max_depth': randint(2, 20),
    'min_samples_split': randint(2, 10),
    'min_samples_leaf': randint(1, 10),
    'max_features': ['sqrt', 'log2', None]
}
random_search = RandomizedSearchCV(
    estimator= RandomForestClassifier(random_state=42),
    param_distributions=param_dist,
    n_iter=500,
    scoring='roc_auc',
    cv=5,
    random_state=42,
    n_jobs=-10
)

random_search.fit(X_train, y_train)

best_params = random_search.best_params_
print("Лучшие параметры:", best_params)

best_model = random_search.best_estimator_
y_pred = best_model.predict(X_test)

print('Лучшие метрики Random Forest при подборе:')
print(f'Accuracy: {accuracy_score(y_test, y_pred)}')
print(classification_report(y_test, y_pred))


Лучшие параметры: {'max_depth': 14, 'max_features': 'log2', 'min_samples_leaf': 9, 'min_samples_split': 7, 'n_estimators': 77}
Лучшие метрики Random Forest при подборе:
Accuracy: 0.7359550561797753
              precision    recall  f1-score   support

       False       0.76      0.72      0.74        94
        True       0.71      0.75      0.73        84

    accuracy                           0.74       178
   macro avg       0.74      0.74      0.74       178
weighted avg       0.74      0.74      0.74       178



/home/simon/anaconda3/lib/python3.12/site-packages/numpy/ma/core.py:2820: RuntimeWarning: invalid value encountered in cast
  _data = np.array(data, dtype=dtype, copy=copy,


<h2>Выводы</h2>

Для кластеризации препаратов по медианному значению <b>IC50</b>, были использованы основные модели классического ML, такие как:

* LogisticRegression;
* DecisionTree;
* Random Forest;
* SVС;
* KNN.

А так же готовое решение CatBoostClassifier.

Наилучшую метрику Accuracy показала модель Random Forest со значением 0.76, что считается не плохим результатом. При этом метрика Recall составляет 0.76 и 0.77 для препаратов со значением IC50 ниже и выше медианного соответственно.

Для подбора наилучших гипер параметров был использован алгоритм случайного поиска. Но ручной подбор гипер параметров оказался лучше.